<a href="https://colab.research.google.com/github/nathalyAlarconT/GenAI_Workshops/blob/main/Intro_to_RAG_Gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup del Notebook

In [ ]:
# IP Asignada a Google Colab
!curl ipecho.net/plain
# https://ipinfo.io/IP

## Instalando las librerías Requeridas

In [ ]:
!pip install --upgrade -q langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub

### Librerías Generales

In [2]:
from google.colab import userdata
import os
from IPython.display import Markdown

### Configurando credenciales

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('SMITH_APIKEY')
GOOGLE_API_KEY = userdata.get('GoogleAIStudio')

### Creación de Folders necesarios

Usaremos dos carpetas:
- Mis Datos, almacenara los PDFs con la info propia
- VectorDB, en esta carpeta se creará la base de datos vectorial

In [4]:
!mkdir /content/MisDatos
!mkdir /content/VectorDB

mkdir: cannot create directory ‘/content/MisDatos’: File exists
mkdir: cannot create directory ‘/content/VectorDB’: File exists


Adiciona los archivos PDF a la carpeta Mis Datos

El archivo utilizado en la demo es:
https://www.lostiempos.com/sites/default/files/edicion_online/las_delicias_de_mi_llajta.pdf




# 1. INDEXING

### Librerías necesarias

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

In [6]:
# @title Configura el origen de los datos
source_data_folder = "/content/MisDatos" # @param {type:"string"}


## Preparando y formateando el contenido necesario

In [7]:
# Leyendo los PDFs del directorio configurado
loader = PyPDFDirectoryLoader(source_data_folder)
data_on_pdf = loader.load()
# cantidad de data cargada
len(data_on_pdf)

20

In [8]:
# Particionando los datos. Con un tamaño delimitado (chunks) y 200 caracters de overlapping para preservar el contexto
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(data_on_pdf)
# Cantidad de chunks obtenidos
len(splits)

38

In [ ]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

## Base de datos vectorial

In [10]:
# @title Configura el path a la base de datos
path_db = "/content/VectorDB" # @param {type:"string"}


In [11]:
# Almacenamos los chunks en la base de datos
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model, persist_directory=path_db)

# 2. RETRIEVAL

## Librerías necesarias

In [12]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## Selección / Configuración del LLM

### Using Gemini

In [13]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

### Using Gemma

In [14]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")
os.environ["KERAS_BACKEND"] = "jax"  # "jax" Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

In [15]:
!pip install -q --upgrade keras-nlp
!pip install --upgrade keras>=3
!pip install -q langchain langchain-google-vertexai

In [17]:
# Load Gemma using LangChain library
from langchain_google_vertexai import GemmaChatLocalKaggle

keras_backend: str = "jax"
model_name = "gemma_1.1_instruct_2b_en"
llm = GemmaChatLocalKaggle(
    model_name=model_name,
    model=model_name,
    keras_backend=keras_backend,
    max_tokens=1024,
)

# If Error on command above, restart the session and run again.

Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to y

In [18]:
# Helpers
from langchain_core.output_parsers import BaseTransformOutputParser
class GemmaOutputParser(BaseTransformOutputParser[str]):
    """OutputParser that parses LLM response and extract
    the generated part."""

    @classmethod
    def is_lc_serializable(cls) -> bool:
        """Return whether this class is serializable."""
        return True

    @property
    def _type(self) -> str:
        """Return the output parser type for serialization."""
        return "gemma_2_parser"

    def parse(self, text: str) -> str:
        """Return the input text with no changes."""
        model_start_token = "model\n"
        idx = text.rfind(model_start_token)
        return text[idx + len(model_start_token) :] if idx > -1 else ""



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)




## Configuración del Prompt y Retriever



In [19]:
retriever = vectorstore.as_retriever()

# https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=6467f92b-9dac-5816-964f-8abcfa4e4456
prompt = hub.pull("rlm/rag-prompt")

In [22]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | GemmaOutputParser()
)

# Ejecución del RAG

In [34]:
# @title Preguntas al Documento
pregunta = "Ingredientes del silpancho. " # @param {type:"string"}
response = rag_chain.invoke(pregunta )
Markdown(response)


The ingredients for the silpancho are chopped onions, tomatoes, lettuce, and rice. The silpancho is prepared by sautéing the vegetables in oil until golden and then

In [42]:
# @title Preguntas al Documento
pregunta = "Name of the dish" # @param {type:"string"}
response = rag_chain.invoke(pregunta )
Markdown(response)


The dish mentioned in the context is huevo duro, a traditional Bolivian traditional dish. It is a savory pancake made with cornmeal, eggs, and potatoes, and is often served with a side of salad.

In [ ]:
# cleanup
# vectorstore.delete_collection()

**Fuentes:**

In [ ]:
# https://dev.to/timesurgelabs/how-to-use-googles-gemini-pro-with-langchain-1eje

# https://python.langchain.com/v0.2/docs/tutorials/rag/

# https://smith.langchain.com/o/6467f92b-9dac-5816-964f-8abcfa4e4456/projects/p/d35fb5ce-7bac-4627-858c-621aa689239f?timeModel=%7B%22duration%22%3A%227d%22%7D